# Multi module Initial channel to channel skew test

# Test Description
This script will test the skew between channels of multiple proteus modules.This script checks the channel-to-channel synchronization by changing the SCLK from 1GHz to 9G with fixed jump (can be change) by reading from a Keysight MSO9254A.

NOTE- This script will compare the skew between following-
1. Channel 1 module 1 and Channel 1  module 2
2. Channel 1 module 1 and Channel 3  module 2
3. Channel 3 module 1 and Channel 3  module 2

As per datasheet maximum possible skew between channels for multimodule is 300psec. 
Test is getting fail if skew is beyond 300psec.


# Test Equipment
1. Tabor Proteus 9484M
2. Keysight Oscilloscope MSO9254A
3. 4 SMA to BNC Cable



# Hardware Connection

    Proteus Connection  ---> Scope Connection

    Module 1 Channel 1 ---> Scope channel 1
    Module 1 Channel 3 ---> Scope channel 2
    Module 2 Channel 1 ---> Scope channel 3
    Module 2 Channel 3 ---> Scope channel 4
    
 

# Test Procedure
1. Connect all the channels of Proteus module with the concerned channel of oscilloscope.
2. Run the script and observe output on scope.



In [1]:
# Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
#matplotlib notebook
import pyvisa as visa
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from scipy import signal
import math
import pdb

In [2]:
## Connect to instrument
connection = 'LAN'
if connection == 'PXI':   
    # not setting this command means DLL are taken from SYS32
    #set_lib_dir_path(r'D:\Projects\ProteusAwg_Anabelle\x64\Debug') 
    # for service connection enter instrument IP
    # for PXI DLL connection enter the module slot number with Auto=False
    # when Auto=True the lowest module will be connected
    inst=connect("6",Auto=False)
    
elif connection == 'LAN':
    proteus_addr = 'TCPIP::192.90.70.22::5025::SOCKET'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
elif connection == 'USB':
    proteus_addr = 'USB0::0x2A8D::0x900E::MY55490134::INSTR'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
elif connection == 'LOCAL':
    proteus_addr = 'TCPIP::127.0.0.1::5025::SOCKET'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
else:
    print("Please select the conection method first")
# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)
inst.default_paranoia_level = 2

Connected to: Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000002232810,1.238.6 --slot#: 2, slot#: 4, 


In [3]:
inst.default_paranoia_level = 2 # Set the default paranoia-level (0, 1 or 2)
inst.send_scpi_cmd('*CLS; *RST') # Clear error-list and reset the instrument
resp = inst.send_scpi_query(":SYST:ERR?")
print(resp)
resp = inst.send_scpi_query('*IDN?') # Get the instrument's *IDN
print('Connected to: ' + resp)
model_name = inst.send_scpi_query('SYST:INF:MODel?') # Get the model:
print('Model: {0} '.format(model_name))

# Get model dependant parameters:
if model_name.startswith('P948'):
    bpp = 2
    dac_mode = 16
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 1
elif model_name.startswith('P908'):
    bpp = 1
    dac_mode = 8
    max_dac = 255
    wpt_type = np.uint8
    offset_factor = 1
else:
    bpp = 2
    dac_mode = 16
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 2

print('DAC waveform format: {} bits-per-point'.format(dac_mode))
multimode = '1'

max_dac = 2 ** dac_mode - 1
half_dac = max_dac / 2.0
print('Max DAC wave-point level: {}'.format(max_dac))
print('Half DAC wave-point level: {}'.format(half_dac))

# Get number of channels
num_channels = int(inst.send_scpi_query(':INST:CHAN? MAX'))
print('Number of channels: {}'.format(num_channels))

# Get the maximal number of segments
max_seg_number = int(inst.send_scpi_query(':TRACe:SELect:SEGMent? MAX'))
print('Max segment number: {}'.format(max_seg_number))

# Get the available memory in bytes of wavform-data (per DDR):
arbmem_capacity = int(inst.send_scpi_query(':TRACe:FREE?'))
print('Available memory per DDR: {:,} wave-bytes'.format(arbmem_capacity))

0, no error
Connected to: Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 
Model: P9484M 
DAC waveform format: 16 bits-per-point
Max DAC wave-point level: 65535
Half DAC wave-point level: 32767.5
Number of channels: 4
Max segment number: 65536
Available memory per DDR: 17,171,480,576 wave-bytes


In [4]:
# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
#inst.send_scpi_cmd(':INST:CHAN 2')
inst.send_scpi_cmd(':TRAC:DEL:ALL') # Delete all segments of the programmable channel's DDR.
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)
# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

0, no error
Number of channels: 4


In [5]:
# Clear and Reset the Modules after selection
inst.send_scpi_cmd(':INST:ACT:SEL 1')
resp = inst.send_scpi_query(":INST:ACT:SEL?")
print("Selected module is" + resp)
inst.send_scpi_cmd('*CLS; *RST')
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)

inst.send_scpi_cmd(':INST:ACT:SEL 2')
resp = inst.send_scpi_query(":INST:ACT:SEL?")
print("Selected module is" + resp)
inst.send_scpi_cmd('*CLS; *RST')
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)

Selected module is1
Connected to: Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 
Selected module is2
Connected to: Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 


In [6]:
scope_addr2=  'USB0::0x2A8D::0x900E::MY55490134::INSTR'

# connect to scope via USB
try:
    resourceManager = visa.ResourceManager()   # Create a connection (session) to the instrument
    scope = resourceManager.open_resource(scope_addr2)
except visa.Error as ex2:
        print('Couldn\'t connect to \'%s\', exiting now...' % scope_addr2)
        sys.exit()
    
    
    
## scope acquisition 
# Send *IDN? and read the response
scope.write('*RST?')
scope.write('*IDN?')
idn = scope.read()
print('*IDN? returned: %s' % idn.rstrip('\n'))

'''
This loop checks the initial skew between different channels on different module
The maximum skew is 200ps.
Input : Channel 1 , Channel 2 , SampleRateJumps
Output: Pass or Fail and if it fails in which Frequncy in fall
'''

highest_mean=0
freq=0

*IDN? returned: KEYSIGHT TECHNOLOGIES,MSO9254A,MY55490134,06.40.01101


In [7]:
MAX_SKEW = 300e-12
InitialFreqSampleRate = 1000
MaximumFreqSampleRate = 9000
FreqSampleRateJump = 1000


test_success = True
for Module1_chan in range(1,4,2):
    for Module2_chan in range(Module1_chan,4,2):
        print( '\nInitial Skew Test between Module 1 channel {0} and Module 2 channel {1} from {2}MHz to {3}MHz in jumps of {4}MHz'.format(Module1_chan, Module2_chan, InitialFreqSampleRate,MaximumFreqSampleRate,FreqSampleRateJump))
        for FreqSampleRate in range (InitialFreqSampleRate,MaximumFreqSampleRate+FreqSampleRateJump,FreqSampleRateJump):
            inst.send_scpi_cmd(':FREQ:RAST {0}MHz'.format(int(FreqSampleRate)))
            FreqSampleRate = int(FreqSampleRate)
            print('\n\t***Test begin for sampling rate {0}MHz ***'.format(FreqSampleRate))
            for i in range(2):
                inst_sel = i+1
                inst.send_scpi_cmd(':INST:ACT {0}'.format(inst_sel))
                resp = inst.send_scpi_query(':SYST:ERR?')
    #             print(resp)
                resp = inst.send_scpi_query(':INST:ACT?')
#                 print('Instrument {} selceted'.format(resp))

            
                if(inst_sel==1):
                    inst.send_scpi_cmd(':INST:CHAN {}'.format(Module1_chan))
                    chan_sel=Module1_chan
                else:
                    inst.send_scpi_cmd(':INST:CHAN {}'.format(Module2_chan))
                    chan_sel=Module2_chan
                        
                        
                        
                        
                        
                # Build waveforms
                seglen = 1024
                cyclelen = seglen
                ncycles = seglen / cyclelen
                waves = [ None for _ in range(3)]
                # square wave
                x = np.linspace(start=0, stop=seglen, num=seglen, endpoint=False)
                y = np.fmod(x, cyclelen)
                y = (y <= cyclelen / 2) * max_dac
                y = np.round(y)
                y = np.clip(y, 0, max_dac)
                waves[2] = y.astype(wpt_type)
#                 plt.plot(x,y)
#                 print("************* WAVEFORM*****************")
                # Download square wave to channel 1
                for ii in range(1):
                    ichan = ii
                    channb = ichan + chan_sel
                    segnum = ii +1
                    wav = waves[2]
#                     print('Download wave to segment {} of channel {}'.format(segnum, channb))   
                    # Select channel
                    inst.send_scpi_cmd(':INST:CHAN {}'.format(channb))    
                    # Define segment
                    inst.send_scpi_cmd(':TRAC:DEF {},{}'.format(segnum, seglen))
                    # Select the segment
                    inst.send_scpi_cmd(':TRAC:SEL {}'.format(segnum))
                #     proteus.send_scpi_cmd(':VOLT:OFFS 0.2')
                #     proteus.send_scpi_cmd(':VOLT 0.5')
                    # Increase the timeout before writing binary-data:
                    inst.timeout = 30000    
                    # Select the segment
                    cmd = ':TRAC:SEL {0}'.format(segnum)
                    inst.send_scpi_cmd(cmd)        
                    # Send the binary-data:
                    inst.write_binary_data(':TRAC:DATA', wav)
                    resp = inst.send_scpi_query(':SYST:ERR?')
                    resp = resp.rstrip()
                    if not resp.startswith('0'):
                        print('ERROR: "{}" after writing binary values'.format(resp))
                    # Play the specified segment at the selected channel:
                    cmd = ':SOUR:FUNC:MODE:SEGM {}'.format(segnum)
                    inst.send_scpi_cmd(cmd)
                    # Turn on the output of the selected channel:
                    inst.send_scpi_cmd(':OUTP ON')    
                    resp = inst.send_scpi_query(':SYST:ERR?')
                                       
                              
                         
    
            inst.send_scpi_cmd(':INST:ACT:SEL 1')
            inst.send_scpi_cmd(':INST:CHAN 1')
            inst.send_scpi_cmd(':XINS:SYNC:FOLL 1')
            inst.send_scpi_cmd(':TRIG:COUPLE ON')
            inst.send_scpi_cmd(':TRIG:CPU:MODE GLOBAL')
    
       
            
            if(Module1_chan==1 and Module2_chan==1):
                scop_channel1=1
                scop_channel2=3
            elif(Module1_chan==1 and Module2_chan==3):
                scop_channel1=1
                scop_channel2=4
            else:
                scop_channel1=2
                scop_channel2=4
                
            
             
            scope.write(':AUToscale')
            time.sleep(5)
         
            scope.write(':TIM:RANG 2E-6')
            scope.write(':CHANnel{}:OFFSet 0'.format(scop_channel1))
            time.sleep(1)
            scope.write(':CHANnel{}:OFFSet 0'.format(scop_channel2))
            time.sleep(1)
            scope.write('CHAN{}:SCAL 0.11'.format(scop_channel1))
            time.sleep(1)
            scope.write('CHAN{}:SCAL 0.06'.format(scop_channel2))
            time.sleep(1)
            scope.write(':MEASure:DELTAtime:DEF RISing,1,MIDD,RISing,1,MIDDle')
            time.sleep(2)
            scope.write(':MEASure:DELTAtime CHANnel{},CHANnel{}'.format(scop_channel1,scop_channel2))
            time.sleep(2)
            scope.write(':MEASure:RESults?')
            time.sleep(2)
            resualt = scope.read()
#             print(resualt)
            mean = float(resualt.split(',')[1])
            tmin = float(resualt.split(',')[2])
            tmax = float(resualt.split(',')[3])
            deviation = float(resualt.split(',')[5])
#             print(abs(mean))
            
            if (abs(mean)<MAX_SKEW):
                scope.write(':TIM:RANG 2E-9')
                time.sleep(1)
                scope.write(':MEASure:DELTAtime:DEF RISing,1,MIDD,RISing,1,MIDDle')
                time.sleep(2)
                scope.write(':MEASure:DELTAtime CHANnel{},CHANnel{}'.format(scop_channel1,scop_channel2))
                time.sleep(2)
                scope.write(':MEASure:RESults?')
                resualt = scope.read()
                time.sleep(1)
#                 print(resualt)
                mean = float(resualt.split(',')[1])
                tmin = float(resualt.split(',')[2])
                tmax = float(resualt.split(',')[3])
                deviation = float(resualt.split(',')[5])
#                 print(abs(mean))
                
               
           
            
            if (abs(mean)>MAX_SKEW): #200e-12 ):
                print('\t***Test Failed in Sample clock: {}MHz, Mean (Average): {:.2f} ps , Range:[{:.2f}]ps , Jitter: (deviation): {:.2f}ps'.format(FreqSampleRate,mean/(10**(-12)),(abs(abs(tmax)-abs(tmin)))/(10**(-12)),deviation/(10**(-12))))
                test_success = False
                #break
            else:
                print('\t***Test Pass in Sample clock: {}MHz, Means(Avrage): {:.2f} ps , Range:[{:.2f}]ps , Jitter: (deviation): {:.2f}ps'.format(FreqSampleRate,mean/(10**(-12)),(abs(abs(tmax)-abs(tmin)))/(10**(-12)),deviation/(100)))
                                 
            if (highest_mean<abs(mean)):
                highest_mean= abs(mean)
                freq= FreqSampleRate
                
            scope.write('*CLS;:DISPlay:CGRade:LEVels ')
            
            for i in range(2):
                inst_sel = i+1
                inst.send_scpi_cmd(':INST:ACT {0}'.format(inst_sel))
                inst.send_scpi_cmd('*CLS; *RST')
                
            for i in range(4):
                scope_chan_sel = i+1
                scope.write(':CHANnel{}:DISPlay OFF'.format(scope_chan_sel))         

                
        if(test_success):
            print('Test Succeed!')
            print('The Highest mean: {} in freq: {} MHz \n'.format(highest_mean,freq))
        else: 
            print('Test Failed!\n' )
                
    
                
# close inst connection
inst.close_instrument()

# Close the connection to the instrument
scope.close()
resourceManager.close()

            
            
            
            
            


Initial Skew Test between Module 1 channel 1 and Module 2 channel 1 from 1000MHz to 9000MHz in jumps of 1000MHz

	***Test begin for sampling rate 1000MHz ***
	***Test Failed in Sample clock: 1000MHz, Mean (Average): 99999900000000006967017440298660512902329124519936.00 ps , Range:[0.00]ps , Jitter: (deviation): 99999900000000006967017440298660512902329124519936.00ps

	***Test begin for sampling rate 2000MHz ***


KeyboardInterrupt: 

In [3]:
#disconnect 
scope.close()
input("Press enter to disconnect")
disconnect()

NameError: name 'scope' is not defined